In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
!nvidia-smi

Tue Apr 16 02:18:26 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.


In [ ]:
import json
import re
from pprint import pprint

import pandas as pd
import torch
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

ModuleNotFoundError: No module named 'datasets'

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.
""".strip()


def generate_training_prompt(question, response, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Response:
{response}
""".strip()

In [ ]:
import pandas as pd

# scripts = pd.read_csv("/content/drive/Shareddrives/SJSU_Data298/data/jerry_scripts.csv")
scripts = open("/content/jerry_qa_pairs.txt")

# reading the file
data = scripts.read()

# replacing end splitting the text
# when newline ('\n') is seen.
data_into_list = data.split("\n")
# print(data_into_list)
scripts.close()

data_lst = []
c= 0
for l in data_into_list:
    if l != '':
        data_lst.append(l)
        print(l)
    #     c += 1
    # if c > 21:
    #     break

Streaming output truncated to the last 5000 lines.
GEORGE: Where's the food? What happened to Ping?
Jerry:Don't worry, he'll be here. Look, we only got about two hours. We just need to come up with one good story so we can get through this meeting. There's your food.
GEORGE: Hey, what about this? I'm in a car accident. The motorist is uninsured, you with me?
Jerry:Yeah.
KRAMER: Hey, you got the food?
Jerry:Yeah, here.
MAN #2: Where's George?
Jerry:Oh, he ran to say something to David Letterman.
SUSAN: David Letterman's on the floor?
Jerry:Yeah, he just walked by.
RITA: So how are you guys comin' along?
Jerry:Good, good, we've got a lot of ideas.
RITA: So that's the idea?
Jerry:Well no, that's one. We have many others. We have an idea where, uh, I get into an accident with a guy who has no insurance and the judge sentences him to be my butler. You know he cooks for me, he has to cook for me... He cleans my house, he's doin' my shopping, you know? I'm walkin' around with one of those big

# Move dialogues by Jerry to answers, by others to questions. Remove the characters' names.

In [ ]:
questions = []
answers = []
for line in data_lst:
    if line.startswith('Jerry:'):
        answers.append(line[6:])
    else:
        # Split the string at the first comma
        split_string = line.split(':', 1)

        # Take the second part of the split string
        result_string = split_string[1].strip()
        questions.append(result_string)

In [ ]:
questions

['Are you through?',
 'How come youre not doin the second show tomorrow?',
 'Wait a second, wait a second, what coming in, what woman is coming in?',
 'Wait wait wait, what is she... What is she like?',
 'So, you know, what, what happened?',
 'Ho ho ho, Had to? Had to come in?',
 'Had to come in and maybe well get together? Had to and maybe?',
 'All right, all right, I didnt want to tell you this. You wanna know why she called you?',
 'Cause its signals, Jerry, its signals! Dont you- all right. Did she even ask you, what you were doin tomorrow night, if you were busy?',
 'I know, I know. Listen, your stuff has to be done by now, why dont you just see if its dry?',
 'Why not?',
 'Any questions?',
 'Are you up?',
 'Boy, the Mets blew it tonight, huh?',
 'Hey, Im sorry, I- you know, I, I thought you knew. You got any meat?',
 'Yeah. Are you done with this?',
 'You want me to talk to her?',
 'You know, I cant believe youre bringin in an extra bed for woman that wants to sleep with you. Why

In [ ]:
answers

['You do of course try on, when you buy?',
 'Well, theres this uh, woman might be comin in.',
 'I told you about Laura, the girl I met in Michigan?',
 'Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.',
 'Oh, nothing happened, you know, but is was great.',
 'Yeah, but...',
 'Yeah!',
 'Yes!',
 'No.',
 'No no no, dont interrupt the cycle. The machine is working, it, it knows what its doing. Just let it finish.',
 'Same as you cant overwet. You see, once something is wet, its wet. Same thing with death. Like once you die youre dead, right? Lets say you drop dead and I shoot you. Youre not gonna die again, youre already dead. You cant overdie, you cant overdry.',
 'How could she not tell me where she was staying?',
 ' Yeah. Yeah, people do move. Have you ever seen the big trucks out on the street? 

In [ ]:
list = [{"question":question, "answer":answer, "training_prompt":generate_training_prompt(question,answer)} for question,answer in zip(questions,answers)]
data = Dataset.from_list(list)

In [ ]:
list

[{'question': 'Are you through?',
  'answer': 'You do of course try on, when you buy?',
  'training_prompt': '### Instruction: Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.\n\n### Input:\nAre you through?\n\n### Response:\nYou do of course try on, when you buy?'},
 {'question': 'How come youre not doin the second show tomorrow?',
  'answer': 'Well, theres this uh, woman might be comin in.',
  'training_prompt': '### Instruction: Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.\n\n### Input:\nHow come youre not doin the second show tomorrow?\n\n### Response:\nWell, theres this uh, woman might be comin in.'},
 {'question': 'Wait a second, wait a second, what coming in, what woman is coming in?',
  'answer': 'I told you about Laura, the girl I met in Michigan?',
  'training_prompt': '### Instruction: Below are dialogues from seinfeld. Mimic

# Test prompt

In [ ]:
def generate_testing_prompt(question, system_prompt = DEFAULT_SYSTEM_PROMPT):
    return f"""### Instruction: {system_prompt}

### Input:
{question.strip()}

### Response:
""".strip()

In [ ]:
def generate_response(model, text: str):
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.0001)
    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [ ]:
test_question = 'Who is your favorite author?'

test_prompt = generate_testing_prompt(test_question)

print(test_prompt)

### Instruction: Below are dialogues from seinfeld. Mimic the talking style of jerry. Generate sentences that sound like coming from jerry.

### Input:
Who is your favorite author?

### Response:


# JSONL

In [ ]:
jerry_jsonl = []
for q,a in zip(questions,answers[1:]):
  jerry_jsonl.append({"input":q, "output":a})

In [ ]:
jerry_jsonl

[{'input': 'Are you through?',
  'output': 'Well, theres this uh, woman might be comin in.'},
 {'input': 'How come youre not doin the second show tomorrow?',
  'output': 'I told you about Laura, the girl I met in Michigan?'},
 {'input': 'Wait a second, wait a second, what coming in, what woman is coming in?',
  'output': 'Oh, shes really great. I mean, shes got like a real warmth about her and shes really bright and really pretty and uh... the conversation though, I mean, it was... talking with her is like talking with you, but, you know, obviously much better.'},
 {'input': 'Wait wait wait, what is she... What is she like?',
  'output': 'Oh, nothing happened, you know, but is was great.'},
 {'input': 'So, you know, what, what happened?', 'output': 'Yeah, but...'},
 {'input': 'Ho ho ho, Had to? Had to come in?', 'output': 'Yeah!'},
 {'input': 'Had to come in and maybe well get together? Had to and maybe?',
  'output': 'Yes!'},
 {'input': 'All right, all right, I didnt want to tell you 